# 指定GPU

## 方式1：指定若干个GPU合并为一个GPU使用
这种方式与多GPU加速不一样，不能加速模型训练，只是为了“扩张”环境的显存。<br>
``
import os
os.environ["CUDA_VISIBLE_DEVICES"]='1,2'
``<br>
或者在终端中运行<br>
`CUDA_VISIBLE_DEVICES=2 python script.py`

## 方式2：指定网络在不同的GPU中
该方式能够指定不同变量在不同的GPU中，同样地不能加速模型训练，只是为了“扩张”环境的显存。与方式一不同的是，该方式能够对每个变量指定GPU位置。<br>
`tf.device('/GPU:1')`

# 显存控制

## 方式1：GPU 显存自动调用
程序根据自身需要自动申请显存空间<br>
``
import tensorflow as tf
import keras.backend.tensorflow_backend as KTB
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.Session(config=config)
KTB.set_session(session)
``

## 方式2：设置程序最大GPU 显存空间
程序只能占据部分GPU显存空间<br>
``
import tensorflow as tf
import keras.backend.tensorflow_backend as KTB
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.5
session = tf.Session(config=config)
KTB.set_session(session)
``

# 多GPU控制

最高支持在8片GPU上并行，需要建立一个原始的模型model，然后生成multi_gpu_model，compile后运行<br>
``
from keras.utils.multi_gpu_utils import multi_gpu_model
mgpu_model = multi_gpu_model(model, gpus=gpu_num)
mgpu_model.compile(...)
``
<br>
该脚本在服务器上运行，终端无法执行ipynb文件。因此，只能讲结果写入log.out中。

In [ ]:
# -*- coding: utf-8 -*-

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from six.moves import xrange

import keras
from keras.datasets import mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()

print('训练集大小: x =', x_train.shape, ', y =', y_train.shape)
print('测试集大小: x =', x_test.shape, ', y =', y_test.shape)

x_train = x_train.reshape(x_train.shape[0], -1).astype('float32') / 255
x_test = x_test.reshape(x_test.shape[0], -1).astype('float32') / 255

# hyper-parameter
image_size = x_train.shape[-1]
training_epoch = 50
num_classes = 10
batch_size = 1000
gpu_num = 4

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

print('training data: x =', x_train.shape, ', y =', y_train.shape)
print('test data: x =', x_test.shape, ', y =', y_test.shape)

from keras.models import Model
from keras.layers import *
from keras.optimizers import *
from keras.activations import *

input_tensor = Input(shape=(image_size,))
fc1 = Dense(units=128, activation=relu)(input_tensor)
output_tensor = Dense(units=num_classes, activation=softmax)(fc1)
model = Model(inputs=input_tensor, outputs=output_tensor)
model.summary()

from keras.utils.multi_gpu_utils import multi_gpu_model

# 最高支持在8片GPU上并行
mgpu_model = multi_gpu_model(model, gpus=gpu_num)
mgpu_model.compile(loss='categorical_crossentropy',
                   optimizer=Adam(1e-3),
                   metrics=['accuracy'])

mgpu_model.fit(x=x_train, y=y_train, batch_size=batch_size, epochs=50)

result = mgpu_model.evaluate(x=x_test, y=y_test)
print(result)